### Library Import

In [1]:
import os
from typing import List, Dict
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
import lightgbm as lgb
from plotly.subplots import make_subplots
import plotly.graph_objects as go


### Data Load

In [2]:
# 파일 호출
data_path: str = "/data/ephemeral/home/BTC/data"
train_df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "train.csv")).assign(_type="train") # train 에는 _type = train 
test_df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "test.csv")).assign(_type="test") # test 에는 _type = test
submission_df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "test.csv")) # ID, target 열만 가진 데이터 미리 호출
df: pd.DataFrame = pd.concat([train_df, test_df], axis=0)

In [3]:
# HOURLY_ 로 시작하는 .csv 파일 이름을 file_names 에 할딩
file_names: List[str] = [
    f for f in os.listdir(data_path) if f.startswith("HOURLY_") and f.endswith(".csv")
]

# 파일명 : 데이터프레임으로 딕셔너리 형태로 저장
file_dict: Dict[str, pd.DataFrame] = {
    f.replace(".csv", ""): pd.read_csv(os.path.join(data_path, f)) for f in file_names
}

for _file_name, _df in tqdm(file_dict.items()):
    # 열 이름 중복 방지를 위해 {_file_name.lower()}_{col.lower()}로 변경, datetime 열을 ID로 변경
    _rename_rule = {
        col: f"{_file_name.lower()}_{col.lower()}" if col != "datetime" else "ID"
        for col in _df.columns
    }
    _df = _df.rename(_rename_rule, axis=1)
    df = df.merge(_df, on="ID", how="left")


100%|██████████| 107/107 [00:03<00:00, 28.39it/s]


### EDA (Explanatory Data Analysis)

### Feature engineering

In [4]:
from data_preprocessing import *

In [5]:
# 모델에 사용할 컬럼, 컬럼의 rename rule을 미리 할당함
cols_dict: Dict[str, str] = {
    "ID": "ID",
    "target": "target",
    "_type": "_type",
    "hourly_market-data_price-ohlcv_all_exchange_spot_btc_usd_close": "close",
    "hourly_market-data_open-interest_all_exchange_all_symbol_open_interest": "open_interest",
    "hourly_network-data_difficulty_difficulty": "difficulty",
    "hourly_network-data_supply_supply_total": "supply_total",
    "hourly_network-data_utxo-count_utxo_count": "utxo_count"
}
df = df[cols_dict.keys()].rename(cols_dict, axis=1)
df.shape

(11552, 8)

In [6]:
# continuous 열을 따로 할당해둠
conti_cols: List[str] = [
    "close",
    "open_interest",
    "difficulty",
    "supply_total",
    "utxo_count"
]

# 최대 24시간의 shift 피쳐를 계산
shift_list = shift_feature(
    df=df, conti_cols=conti_cols, intervals=[_ for _ in range(1, 24)]
)

# concat 하여 df 에 할당
df = pd.concat([df, pd.concat(shift_list, axis=1)], axis=1)

In [7]:
# _type에 따라 train, test 분리
train_df = df.loc[df["_type"]=="train"].drop(columns=["_type"])
test_df = df.loc[df["_type"]=="test"].drop(columns=["_type"])

### Model Training

In [8]:
def close_to_class(series: pd.Series) -> pd.Series:
    """close 변수를 target값으로 변환하는 함수입니다.

    Args:
        series (pd.Series): 변환을 원하는 close 변수

    Returns:
        pd.Series: 변환된 target 값
    """
    close = pd.DataFrame()
    close['close'] = series
    close['close_lag1'] = close['close'].shift(1)
    close['close_lag1_percent'] = (close['close'] - close['close_lag1']) / close['close_lag1']
    close['class'] = close['close']
    for i in range(close.shape[0]):
        if close.loc[i, 'close_lag1_percent'] < -0.005:
            close.loc[i, 'class'] = 0
        elif close.loc[i, 'close_lag1_percent'] < 0:
            close.loc[i, 'class'] = 1
        elif close.loc[i, 'close_lag1_percent'] < 0.005:
            close.loc[i, 'class'] = 2
        else:
            close.loc[i, 'class'] = 3
            
    return close['class']

In [9]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings("ignore")

def model_train(
    model,
    X_train,
    y_train,
    cv: int = 5,
    metric: str = "accuracy"
) -> tuple[None, float]:
    """모델을 불러와 cross validation을 이용해 예측 성능을 확인하는 함수입니다.

    Args:
        model (Any): 학습을 수행할 모델
        X_train (Any): 훈련 데이터 피처
        y_train (Any): 훈련 데이터
        cv (int): KFold 검정 횟수
        metric (str): 성능 지표. accuracy(target), mae, mape mse 중 선택.

    Returns:
        _type_: _description_
    """
    kfold = KFold(n_splits=cv)
    score_list = []
    for train_index, valid_index in kfold.split(X_train):
        X_train_fold, y_train_fold = X_train.iloc[train_index], y_train.iloc[train_index]
        X_valid, y_valid = X_train.iloc[valid_index], y_train.iloc[valid_index]

        valid_target = X_valid["target"]
        X_train_fold.drop("target", axis=1, inplace=True)
        X_valid.drop("target", axis=1, inplace=True)

        # preprocessing
        X_train_fold.fillna(X_train_fold.mean(), inplace=True)
        y_train_fold.fillna(y_train_fold.mean(), inplace=True)
        X_valid.fillna(X_valid.mean(), inplace=True)
        y_valid.fillna(y_valid.mean(), inplace=True)  # 이 부분을 mice와 같은 방법으로 조정할 예정. feature selection 등도 여기에서.

        model.fit(X_train_fold, y_train_fold)
        y_pred = model.predict(X_valid)
        score = evaluate(valid_target, y_valid, y_pred, metric=metric)
        score_list.append(score)
    
    return np.mean(score_list)

def evaluate(valid_target, y_valid, y_pred, metric):
    if metric == 'accuracy':
        classes = close_to_class(y_pred)
        return accuracy_score(valid_target, classes)
    if metric == 'mae':
        return mean_absolute_error(y_valid, y_pred)
    if metric == "mse":
        return mean_squared_error(y_valid, y_pred)
    if metric == "mape":
        return mean_absolute_percentage_error(y_valid, y_pred)

In [11]:
X_train = train_df.drop(["ID", "close"], axis=1)
y_train = train_df["close"]
X_test = test_df.drop(["ID", "close"], axis=1)
y_test = test_df["close"]
X_test.fillna(X_test.mean(), inplace=True)
y_test.fillna(y_test.mean(), inplace=True)

lr_reg = LinearRegression()
ridge_reg = Ridge(random_state=42)
lasso_reg = Lasso(random_state=42)
lr_score = model_train(lr_reg, X_train, y_train, cv=5, metric="accuracy")
ridge_score = model_train(ridge_reg, X_train, y_train, cv=5, metric="accuracy")
lasso_score = model_train(lasso_reg, X_train, y_train, cv=5, metric="accuracy")
print(f"{lr_reg}: {lr_score}")
print(f"{ridge_reg}: {ridge_score}")
print(f"{lasso_reg}: {lasso_score}")

LinearRegression(): 0.3742009132420091
Ridge(random_state=42): 0.373972602739726
Lasso(random_state=42): 0.3700913242009133


### Inference (Linear Regression)

In [22]:
# lr predict
y_test_pred = lr_reg.predict(X_test.drop("target", axis=1))
y_test_pred_class = close_to_class(y_test_pred)

In [23]:
# lr output
submission_df = submission_df.assign(target = y_test_pred_class)
submission_df["target"] = submission_df["target"].astype(np.int8)
submission_df.to_csv("output.csv", index=False)

# Inference ()

In [ ]:
# lgb predict
y_test_pred = lgb_model.predict(test_df.drop(["target", "ID"], axis = 1))
y_test_pred_class = np.argmax(y_test_pred, axis = 1)

### Output File Save

In [23]:
# output file 할당후 save 
submission_df = submission_df.assign(target = y_test_pred_class)
submission_df.to_csv("output.csv", index=False)
